In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
from urllib import urlopen
from bs4 import BeautifulSoup
import os
import requests
import json
import pickle

'''this program will take around an hour to run'''

In [ ]:
no_hist_symbol = []

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)      
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)    
    return tickers    

def select_ticker(tick):
    url = 'https://finance.yahoo.com/quote/{0}/analysts?p={0}'
    return url.format(tick)

def yahoo_finance_analyst_data(url):
    req = urlopen(url)
    raw = req.read()
    soup = BeautifulSoup(raw,'lxml')
    return soup

def auto_symbols(ticker):
    url = select_ticker(ticker)
    return url

def grab_dict(soup):
    l = [i for i in soup.find_all('script')]
    mylist = [str(item) for item in l]
    m = max(mylist,key=len)
    ind = mylist.index(m)
    data = l[mylist.index(m)]
    data_dict = data.text[112:-12]
    all_tables = json.loads(data_dict)
    return all_tables

def upgrade_downgrade_pandas(dic,tick):
    try:
        stock_upgrade_downgrade_history = dic['context']['dispatcher']['stores']['QuoteSummaryStore']['upgradeDowngradeHistory']['history']
        pd_stock_upgrade_downgrade_history = pd.DataFrame(stock_upgrade_downgrade_history)
        pd_stock_upgrade_downgrade_history['epochGradeDate'] = pd.to_datetime(pd_stock_upgrade_downgrade_history['epochGradeDate'],unit='s')
        pd_stock_upgrade_downgrade_history.columns.values[1] = 'date'
        pd_stock_upgrade_downgrade_history.insert(0,'stockTicker',tick)
        return pd_stock_upgrade_downgrade_history
    except Exception:
        print "error" + tick
        no_hist_symbol.append(tick)
        

def single_company_hist(tick):
    url = select_ticker(tick)
    soup = yahoo_finance_analyst_data(url)
    full_dict = grab_dict(soup)
    return upgrade_downgrade_pandas(full_dict,tick)

def list_of_all_stock_grading_history(good_list):
    
    all_stock_grading_history = []
    for ticker in good_list:
        if ticker not in no_hist_symbol:
            url = auto_symbols(ticker)
            soup = yahoo_finance_analyst_data(url)
            all_tables = grab_dict(soup)
            grading_history = upgrade_downgrade_pandas(all_tables,ticker)
            all_stock_grading_history.append(grading_history)
    concat_lists_to_pd = pd.concat(all_stock_grading_history)
    return concat_lists_to_pd

def all_hist_to_tick(df):
    list_of_stock_histories = []
    for ticker in df['stockTicker'].unique():
        try:
            stock_df = pdr.get_data_yahoo(symbols=ticker, start=df.loc[df['stockTicker']==ticker].date.iloc[-1])
            stock_df['stockTicker'] = ticker
            stock_df.reset_index(inplace = True)
            stock_df['Percent_change'] =  stock_df.Close.pct_change()
            list_of_stock_histories.append(stock_df)
            print ticker
        except Exception: 
            print 'error @ ' + ticker
    concat_lists_to_pd = pd.concat(list_of_stock_histories)
    return concat_lists_to_pd

def combine_toGrade(df):
    
    Strong_Buy = ['Outperform','Mkt Outperformer','Recomm List','Market Outperform','Mkt Outperform','NT Strong Buy','Recommended List','LT Accumulate','Outperformer','Top Pick', 'Strong Buy','Outperf. Signif.','Sector Outperform','Conviction Buy']
    Buy = ['Buy','Overweight','Trading Buy','NT Accum/LT Buy', 'NT Ntrl/LT Buy','Long-Term Buy','Above Average','LT Attractive','LT Accum','LT Buy','Accumulate','NT Accum','NT Nuet/ LT Buy','Attractive','Positive','Long-term Buy','Add','NT Buy']
    Hold = ['Hold','Neutral','Peer perform','Equal-weight','Perform-In-Line','Perform In Line','In-line','Market Perform','Mkt Performer','Average','NT Neutral','Maintain Position','ST Neutral','Sector Perform','Mixed','Market Weight','Equal-weight','Perform','Sector Weight','Peer Perform','In-Line','Fair Value','Mkt Perform','Equal Weight']
    Underperform = ['Underperform', 'Underperformer','Below Average','Unattractive','Market Underperform','Cautious','Mkt Underperform','Sector Underperform']
    Sell = ['Sell','Underweight','Reduce','Negative','Trim']

    list_dict = {
    'Strong_buy':Strong_Buy,
    'Buy':Buy,
    'Hold':Hold,
    'Underperform':Underperform,
    'Sell':Sell
    }
    reversed_dict = {val: key for key in list_dict for val in list_dict[key]}
    df['toGrade_combined'] = df['toGrade'].map(reversed_dict)
    df['fromGrade_combined'] = df['fromGrade'].map(reversed_dict)
    return df

#symbols=['SPY']
def get_stock_data(symbols):
    for symbol in symbols:
        print symbol
        df_temp =  pdr.get_data_yahoo(symbols=symbol)
        df_temp = df_temp.rename(columns={'Adj Close':symbol})
        return df_temp

def use_only_spy_dates(spy,df):
    df_list = []
    for symbol in df.stockTicker.unique():
        dates = pd.DataFrame(index=spy)
        temp_df=df.loc[df['stockTicker'] == symbol]
        dates = dates.join(temp_df)
        df_list.append(dates)
    return pd.concat(df_list)

        

def merge_on_spy_dates(df,dates):
    firms = []
    for firm in df['firm'].unique():
        df1 = df.loc[df['firm']==firm]
        temp_list = []
        for symbol in df1['stockTicker'].unique():
            temp_df = df1.loc[df1['stockTicker']==symbol]
            dates_df = dates.loc[dates['stockTicker']==symbol]
            #df_dates = df_dates.join(temp_df)
            dates_df=dates_df.set_index(['stockTicker','Adj Close'],append=True).join(temp_df.set_index(['stockTicker','Adj Close'],append=True)).reset_index(level=['stockTicker','Adj Close'])

            temp_list.append(dates_df)
        firms.append(pd.concat(temp_list))
        print firm
    return pd.concat(firms)
        



if __name__=='__main__':
    print "get snp500 stock tickers\n"
    snp500tickers = save_sp500_tickers()
    print "use tickers to grab stock upgrade downgrade history\n"
    stock_to_grade = list_of_all_stock_grading_history(snp500tickers)
    print "use tickers to grab all stock price data\n"
    all_snp_price_hist = all_hist_to_tick(stock_to_grade)
    print "combine like-terms of grading into a minimalized list\n"
    stock_to_grade = combine_toGrade(stock_to_grade)
    
    #make copies of dataframes
    stock_to_grade_copy = stock_to_grade.copy()
    all_snp_price_hist_copy = all_snp_price_hist.copy()
    spy_stock_hist_data = all_snp_price_hist.copy()
    
    #remove unneeded columns from the dataframes
    spy_stock_hist_data = spy_stock_hist_data[['Date','stockTicker','Adj Close']]
    stock_to_grade_copy = stock_to_grade_copy[['date','stockTicker','firm','toGrade_combined']]
    all_snp_price_hist_copy = all_snp_price_hist_copy[['Date', 'Adj Close','stockTicker']]
    
    #set the Date as index
    stock_to_grade_copy=stock_to_grade_copy.set_index('date')
    stock_to_grade_copy.index.names=['Date']
    all_snp_price_hist_copy=all_snp_price_hist_copy.set_index('Date')
    
    #change the date index to datetime object
    all_snp_price_hist_copy.index.to_datetime()
    stock_to_grade_copy.index.to_datetime()
    
    print "merge grading df and price df on Date and stockTicker\n"
    stock_price_merge_grade = stock_to_grade_copy.set_index('stockTicker',append=True).join(all_snp_price_hist_copy.set_index('stockTicker',append=True)).reset_index(level='stockTicker')
    #rearange order of columns
    cols = ['Adj Close','stockTicker','firm','toGrade_combined']
    stock_price_merge_grade = stock_price_merge_grade[cols]
    
    print "grab dates of SPY stock ticker to serve as a constant\n"
    spy_dates = get_stock_data(['SPY']).index
    spy_dates_symb = use_only_spy_dates(spy_dates,all_snp_price_hist_copy)
    
    print "create a concatenated joined dataframe for each firms upgrade and downgrade history with stockTicker prices before and after included\n"
    print "this step will take around 20 minutes"
    final_df = merge_on_spy_dates(stock_price_merge_grade,spy_dates_symb)
    
    print "write to csv titled: snp500finaldf.csv" 
    final_df.to_csv("snp500finaldf.csv")

get snp500 stock tickers

use tickers to grab stock upgrade downgrade history

errorBRK.B
errorBF.B
